### TODO
* positionSnapshots (suppl. accounts and positions)
* dailyMarketSnapshots (suppl. markets)
* events: deposits, withdraws, borrows, repays, liquidates, transfers, flashloans

In [1]:
import requests
import pandas as pd
import numpy as np
import time
import pickle
import json
import os
import random

In [2]:
def to_timestamp(date):
    return int(date.timestamp())

In [3]:
def run_query(api, query): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post(api, json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))

In [4]:
def get_query_string(start_time, end_time, skip=0):
    query = '''{
      positionSnapshots(
        where: {timestamp_gt: ''' + str(start_time) + ''', timestamp_lt: ''' + str(end_time) + '''}
        first: 1000
        skip: ''' + str(skip) + ''' 
        orderBy: timestamp
      ) {
        account {
          id
        }
        position {
          id
          asset {
            id
            symbol
          }
          market {
            id
            name
          }
          isCollateral
          isIsolated
          side
          type
          _eMode
        }
        balance
        balanceUSD
        blockNumber
        hash
        id
        index
        logIndex
        nonce
        principal
        timestamp
      }
    }'''
    return query

# For Compound V3
def get_query_string(start_time, end_time, skip=0):
    query = '''{
      positionSnapshots(
        where: {timestamp_gt: ''' + str(start_time) + ''', timestamp_lt: ''' + str(end_time) + '''}
        first: 1000
        skip: ''' + str(skip) + ''' 
        orderBy: timestamp
      ) {
        account {
          id
        }
        position {
          id
          asset {
            id
            symbol
          }
          market {
            id
            name
          }
          isCollateral
          isIsolated
          side
          type
        }
        balance
        balanceUSD
        blockNumber
        hash
        id
        index
        logIndex
        nonce
        principal
        timestamp
      }
    }'''
    return query

# For Compound V2
def get_query_string(start_time, end_time, skip=0):
    query = '''{
      positionSnapshots(
        where: {timestamp_gt: ''' + str(start_time) + ''', timestamp_lt: ''' + str(end_time) + '''}
        first: 1000
        skip: ''' + str(skip) + ''' 
        orderBy: timestamp
      ) {
        balance
            blockNumber
            hash
            id
            logIndex
            nonce
            timestamp
            position {
              id
              isCollateral
              side
              account {
                id
              }
              market {
                id
                name
              }
            }
          }
    }'''
    return query

In [5]:
def query_snapshots(api, start_time, end_time):
    all_results = list()
    for i in range(0, 6000, 1000):
        query = get_query_string(start_time, end_time, skip=i)
        result = run_query(api, query)['data']['positionSnapshots']
        all_results.extend(result)
        if len(result) < 1000: break
    return all_results

In [6]:
d = dict()
d['AAVE V2 Ethereum'] = ('https://api.thegraph.com/subgraphs/name/messari/aave-v2-ethereum', '2020-11-30')
d['AAVE V3 Ethereum'] = ('https://api.thegraph.com/subgraphs/name/messari/aave-v3-ethereum', '2023-01-27')
d['Compound V2 Ethereum'] = ('https://api.thegraph.com/subgraphs/name/messari/compound-v2-ethereum', '2019-05-07')
d['Compound V3 Ethereum'] = ('https://api.thegraph.com/subgraphs/name/messari/compound-v3-ethereum', '2022-08-26')

In [7]:
subgraph = 'AAVE V2 Ethereum'
api = d[subgraph][0]
folder = f'Messari/{subgraph}/positionSnapshots'

In [8]:
#start_date, end_date = d[subgraph][1], '2024-01-01'
start_date, end_date = '2023-11-01', '2024-01-01'
all_dates = pd.date_range(start=start_date, end=end_date).tolist()
all_dates = list(map(lambda date: int(date.timestamp()), all_dates))

In [9]:
all_dates_stack = all_dates.copy()[::-1]
while len(all_dates_stack) > 1:
    (start_time, end_time) = int(all_dates_stack.pop()), int(all_dates_stack[-1])
    
    if os.path.isfile(f'{folder}/positionSnapshots_{start_time}_{end_time}.json'):
        continue
    
    temp = query_snapshots(api, start_time, end_time)
    
    if len(temp) >= 6000:
        print("Extend", start_time, end_time, len(temp))
        all_dates_stack.extend(np.linspace(int(start_time), int(end_time), num=5)[:-1][::-1])
        continue
    
    print("Write", start_time, end_time, len(temp))
    with open(f'{folder}/positionSnapshots_{start_time}_{end_time}.json', 'w') as outfile:
        outfile.write(json.dumps(temp))